In [11]:
#Importing libraries
import pandas as pd
from google.cloud import bigquery
import json
import os 
import re
import time
import datetime as dt

In [2]:
#Obtain path to BQ credentiasls.
with open('path.txt', 'r') as f:
    path = f.read()
#print(path)

In [3]:
#Using credentials to connect to BQ
# Set enviroment variable GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=path

In [4]:
#Let's initialize a bq client.
#This dataset comes from a BQ-DB. Project's name is power-bi*
BigQuery_client = bigquery.Client()
query="""
#standardSQL
SELECT * FROM `power-bi-326113.analytics_264987177.events_*`
"""
#Save results in query_results
query_results = BigQuery_client.query(query)

In [5]:
#Transform query_results to a pandas dataframe.
df = pd.read_gbq(query)
print(df.head(5))

  event_date   event_timestamp     event_name  \
0   20210929  1632933787481517  session_start   
1   20210929  1632933787481517      page_view   
2   20210929  1632933795844292         scroll   
3   20210929  1632943150430016    first_visit   
4   20210929  1632943150430016  session_start   

                                        event_params  \
0  [{'key': 'ga_session_id', 'value': {'string_va...   
1  [{'key': 'entrances', 'value': {'string_value'...   
2  [{'key': 'ga_session_number', 'value': {'strin...   
3  [{'key': 'ga_session_id', 'value': {'string_va...   
4  [{'key': 'engaged_session_event', 'value': {'s...   

   event_previous_timestamp  event_value_in_usd  event_bundle_sequence_id  \
0                       NaN                 NaN                 106444205   
1                       NaN                 NaN                 106444205   
2                       NaN                 NaN                 114806980   
3                       NaN                 NaN             

In [6]:
#So, that was the hard part. Now, we need to clean this dataset and get some important data such as city and country.
#I'm going to use a list comprehension to do that, then I'll use some high level functions like map. I think both are very useful and very helpful to learn.
df['city'] = [x['city'] for x in df['geo']]
#df['country'] = [x['geo']['country'] for x in df['geo']]
df['country'] = df['geo'].map(lambda x: x['country'])
#Source of data.
df['source'] = df['traffic_source'].map(lambda x: x['source'])



In [7]:
df.head(5)

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,city,country
0,20210929,1632933787481517,session_start,"[{'key': 'ga_session_id', 'value': {'string_va...",NaN,NaN,106444205,NaN,None,1613402758.1632402055,...,"{'continent': 'Americas', 'country': 'Argentin...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",2366041961,WEB,None,None,[],Moron,Argentina
1,20210929,1632933787481517,page_view,"[{'key': 'entrances', 'value': {'string_value'...",NaN,NaN,106444205,NaN,None,1613402758.1632402055,...,"{'continent': 'Americas', 'country': 'Argentin...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",2366041961,WEB,None,None,[],Moron,Argentina
2,20210929,1632933795844292,scroll,"[{'key': 'ga_session_number', 'value': {'strin...",NaN,NaN,114806980,NaN,None,1613402758.1632402055,...,"{'continent': 'Americas', 'country': 'Argentin...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",2366041961,WEB,None,None,[],Moron,Argentina
3,20210929,1632943150430016,first_visit,"[{'key': 'ga_session_id', 'value': {'string_va...",NaN,NaN,879458112,NaN,None,464139976.1632943144,...,"{'continent': 'Americas', 'country': 'Argentin...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",2366041961,WEB,None,None,[],Buenos Aires,Argentina
4,20210929,1632943150430016,session_start,"[{'key': 'engaged_session_event', 'value': {'s...",NaN,NaN,879458112,NaN,None,464139976.1632943144,...,"{'continent': 'Americas', 'country': 'Argentin...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",2366041961,WEB,None,None,[],Buenos Aires,Argentina


In [13]:
#We need page_location and I made this algorithm to obtain it.
lista= []
#print(df.event_params[0])
for i in df.event_params:
        for j in i:
                if j['key']=='page_location':
                        lista.append(j['value']['string_value'])

In [15]:
lista2 = [re.search('hexaId=(.{6})', i).group(1) for i in lista if re.search('hexaId=(.{6})', i)!=None]
lista3 = []
operators = []
for i in lista:
    try:
        if re.search('hexaId=(.{6})', i)!=None:
            lista3.append(re.search('hexaId=(.{6})', i).group(1)).replace('&', '')
        else:
            lista3.append('No tiene hexaId')
    except Exception as e:
        print('The operation was failed. ', e)

    try:
        if re.search('(?<=id_operador=)[A-Z]*',i)!=None:
            operators.append(re.search('(?<=id_operador=)[A-Z]*',i).group(0))
        else:
            operators.append('Sin operador')

    except Exception as e:
        print('The operation was failed. ', e)

In [16]:
#Assign values that we found earlier.
df['hexaId'] = lista3
df['operator'] = operators
#Working with time and formatting events.
#First, we transform timestamp into datetime format.
df['user_first_touch_timestamp']=pd.to_datetime(df['user_first_touch_timestamp'], unit='us')
df['event_timestamp']=pd.to_datetime(df['event_timestamp'], unit='us')
#Then, we can format as we need.
df['user_first_touch_timestamp']=df[df['user_first_touch_timestamp'].isna()==False]['user_first_touch_timestamp'].map(lambda x: dt.datetime.strftime(x,'%Y/%m/%d %H:%M:%S' ))
df['event_timestamp']=df['event_timestamp'].map(lambda x: dt.datetime.strftime(x,'%Y/%m/%d %H:%M:%S' ))

In [17]:
#We need mobile_bran_name and device_category.
df['mobile_brand_name'] =[i['mobile_brand_name'] for i in df.device ]
df['device_category'] = [i['category'] for i in df.device ]
#Also, we need to convert hexaid to a decimal value.
df['id'].loc[df['hexaId']=='No tiene hexaId']=0
df2 =df[['hexaId', 'event_date','event_timestamp' ,'event_name','country' ,'city', 'mobile_brand_name', 'device_category', 'operator', 'source', 'id', 'user_first_touch_timestamp']]
df3=df2
#df3 = df2[df2['hexaId']!='Sin hexaid']
print(df3.head(5))

In [19]:
df2 =df[['hexaId', 'event_date','event_timestamp' ,'event_name','country' ,'city', 'mobile_brand_name', 'device_category']]
df3 = df2[df2['hexaId']!='No tiene hexaId']
print(df3.head(5))

   hexaId event_date   event_timestamp       event_name    country  \
34  F63EA   20210929  1632923634009950        page_view  Argentina   
35  F63EA   20210929  1632923797856531  user_engagement  Argentina   
36  3CC4C   20210929  1632923875211153        page_view  Argentina   
37  3CC4C   20210929  1632924610579964  user_engagement  Argentina   
38  3D54B   20210929  1632924639205002        page_view  Argentina   

             city mobile_brand_name device_category  
34  Vicente Lopez            Google         desktop  
35  Vicente Lopez            Google         desktop  
36  Vicente Lopez            Google         desktop  
37  Vicente Lopez            Google         desktop  
38  Vicente Lopez            Google         desktop  


In [20]:
df3.hexaId.value_counts()

713BD    49
1537F    40
C0864    24
3E9DB    23
1A951    19
         ..
3E85A     1
3C18D     1
701E5     1
1A464     1
1A813     1
Name: hexaId, Length: 257, dtype: int64